# All shortest paths

All shortest path algorithm implemented by QUBO in quantum annealer. Classical version of this is for example bellman-ford: https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for defined $(s,t) \in E^2$.

Solution is inspired by "Directed Edge-Based Approach" from Kraus & McCollum (2020) Solving the Network Shortest Path Problem on a Quantum Annealer: https://doi.org/10.1109/TQE.2020.3021921

Penalty $p=\sum w_{xy}$.

Following constraints are build to QUBO: 
1. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
2. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
3. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
4. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
5. Pathing having lower weights should be prioritised

Proper path would give energy level of $w-p$ (constraint 4 brings minimum $p$ and constraint 1 and 2 both minimum $-p$). From this set of samples we choose lowest energy level sample. All samples with energy level below zero are correct pahts between $(s,t)$.

In [40]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## 1. First run with simple graph

### Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [41]:
E = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5), (3, 4, 3), (2, 4, 8)])
vertices = 5
s = 0
t = 4

Above graph visualised:

![](graph4.png)

In [42]:
#E = np.array([(0, 1, 8), (1, 3, 5), (0, 2, 2), (2, 1, 4), (2, 4, 7), (4, 3, 3)])
#vertices = 5
#s = 0
#t = 3

Above graph visualised:
    
![](graph2.png)

### Construct max_count and labels variables

In [43]:
max_count = 0
labels = {}
for i,e in enumerate(E):
    max_count += e[2]
    labels[i] = str(e[0]) + '-' + str(e[1])
print('Max count:',max_count)

Max count: 26


### Some helper functions for getting results from sampleset

In [44]:
def path_from_sample(sample,s,t,E):
    w = 0
    i = s
    path = str(i)
    while i!=t:
        found = False
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path += '-'+str(i)
                w += e[2]
                found = True
        if not found:
            print('Path broken')
            break
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset,s,t,E):
    energy= sampleset.first.energy
    ss = sampleset.filter(lambda s: s.energy==energy)
    res = []
    for sample in ss:
        st, path, w = path_from_sample(sample,s,t,E)
        if st not in res:
            res.append((st,path,w))
    return res

### Function to create QUBO

Constraints: 
1. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
2. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
3. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
4. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
5. Pathing having lower weights should be prioritised

In [45]:
def create_qubo(E,p,s,t):
    edges = len(E)
    Q = np.zeros((edges,edges))

    # Constraint 1
    for i in range(edges):
        if E[i][1]==s:
            Q[i][i] += p
        if E[i][0]==s:
            Q[i][i] -= p
        
    # Constraint 2
    for i in range(edges):
        if E[i][0]==t:
            Q[i][i] += p
        if E[i][1]==t:
            Q[i][i] -= p

    # Constraint 3
    for i in range(edges):
        for j in range(i+1,edges):
            if E[i][0]==E[j][0] or E[i][1]==E[j][1]:
                Q[i,j] = 2*p

    # Constraint 4
    for i in range(edges):
        Q[i,i] += p
        for j in range(i+1,edges):
            if E[i][1]==E[j][0] or E[i][0]==E[j][1]:
                Q[i,j] -= p

    # Constraint 5
    for i,e in enumerate(E):
        Q[i][i] += e[2]        

    return Q

### Create QUBO

In [46]:
t1 = time.time()
Q = create_qubo(E,max_count,s,t)
qubo_time = (time.time()-t1)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))
print(labels)
print(Q)

Time used for construction Q (ms): 0.369

{0: '0-2', 1: '2-1', 2: '1-3', 3: '3-2', 4: '0-1', 5: '3-4', 6: '2-4'}
[[  1. -26.   0.  52.  52.   0. -26.]
 [  0.  28. -26. -26.  52.   0.  52.]
 [  0.   0.  29. -26. -26. -26.   0.]
 [  0.   0.   0.  30.   0.  52. -26.]
 [  0.   0.   0.   0.   5.   0.   0.]
 [  0.   0.   0.   0.   0.   3.  52.]
 [  0.   0.   0.   0.   0.   0.   8.]]


### Create BQM from QUBO

In [47]:
t1 = time.time()
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm_time = (time.time()-t1)*1000
bqm = bqm.relabel_variables(labels, inplace=False)

### Local deterministic classical solver

In [48]:
t1 = time.time()
sampleset = dimod.ExactSolver().sample(bqm)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
energy= sampleset.first.energy
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 1.867

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc.
1   0   1   1   1   0   0   1  -17.0       1
3   0   1   0   0   1   0   0  -17.0       1
0   1   0   1   0   0   0   1  -15.0       1
2   1   0   1   0   1   1   0   -6.0       1
['BINARY', 4 rows, 4 samples, 7 variables]


In [49]:
for r in result_info(sampleset,s,t,E):
    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

Route 0-4: 0-2-1-3-4, weight 9
Route 0-4: 0-2-4, weight 9


### Local heuristic classical solver

In [11]:
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2)

Time used (ms): 65.236

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc.
0   0   1   0   0   1   0   0  -17.0     190
3   0   1   1   1   0   0   1  -17.0     171
1   1   0   1   0   0   0   1  -15.0     128
2   1   0   1   0   1   1   0   -6.0      11
['BINARY', 4 rows, 500 samples, 7 variables]


In [12]:
for r in result_info(sampleset2,s,t,E):
    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

Route 0-4: 0-2-4, weight 9
Route 0-4: 0-2-1-3-4, weight 9


### Quantum solver

In [13]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [14]:
sampleset3 = EmbeddingComposite(machine).sample(bqm, num_reads=500)

In [38]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0).aggregate())

QPU time used (ms): 66.6
Physical qubits used: 8

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc. chain_.
0   0   1   0   0   1   0   0  -17.0     100     0.0
1   0   1   1   1   0   0   1  -17.0     101     0.0
2   1   0   1   0   0   0   1  -15.0      57     0.0
3   1   0   1   0   1   1   0   -6.0      29     0.0
['BINARY', 4 rows, 287 samples, 7 variables]


In [50]:
for r in result_info(sampleset3.aggregate(),s,t,E):
    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

Route 0-4: 0-2-4, weight 9
Route 0-4: 0-2-1-3-4, weight 9


In [17]:
dwave.inspector.show(sampleset3)

Serving Inspector on http://127.0.0.1:18001/?problemId=ce5ef1f4-6028-46bd-be0f-5e064985c00c

'http://127.0.0.1:18001/?problemId=ce5ef1f4-6028-46bd-be0f-5e064985c00c'

### Hybrid solver

Hybrid solver brings only one solution, not good here

In [18]:
#sampleset4 = LeapHybridSampler().sample(bqm)

In [19]:
#hyb_time = sampleset4.info['qpu_access_time'] / 1000
#print('QPU time used (ms): {:.1f}\n'.format(hyb_time))
#print(sampleset4) 

In [20]:
#for r in result_info(sampleset4,s,t):
#    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

### Timings

In [21]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.131
Construting BQM: 0.380

Local deterministic solver: 0.9
Local heuristic solver: 65.2
Quantum solver: 66.6


## 2. Second more complex graph

In [22]:
import networkx as nx
import random

Parameters with several equal lengths: maxvert=15 s=0 t=9; maxvert=10 s=1 t=9

In [59]:
seed = 42
maxvert = 15
random.seed(seed)
G = nx.gnp_random_graph(maxvert, 0.20, seed)
G = G.to_directed()
nx.set_edge_attributes(G, {e: {'weight': random.randint(1, 10)} for e in G.edges})

vertices = len(G.nodes)
s = 0
t = 9

In [60]:
E = [] 
for e in G.edges(data=True):
    E.append((e[0],e[1],e[2]['weight']))
#    print(e)
print('Number of edges:',len(E))
print('Number of vertices:',vertices)

Number of edges: 40
Number of vertices: 15


### Test with classical algorithm

In [61]:
t1 = time.time()
res = nx.all_shortest_paths(G,s,t,weight='weight')
classical_time = (time.time()-t1)*1000
print('Time used by classical algorithm (ms): {:.3f}\n'.format(classical_time))
for r in res:
     print(r)

Time used by classical algorithm (ms): 0.168

[0, 14, 9]
[0, 10, 9]
[0, 13, 11, 3, 9]


In [62]:
max_count = 0
labels = {}
for i,e in enumerate(E):
    max_count += e[2]
    labels[i] = str(e[0]) + '-' + str(e[1])
print('Max count:',max_count)

Max count: 191


### QUBO and BQM

In [63]:
t1 = time.time()
Q = create_qubo(E,max_count,s,t)
qubo_time = (time.time()-t1)
print('Time used for construction Q (s): {:.3f}\n'.format(qubo_time))

Time used for construction Q (s): 0.001



In [64]:
t1 = time.time()
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm_time = (time.time()-t1)
bqm = bqm.relabel_variables(labels, inplace=False)
print('Time used for construction BQM (s): {:.3f}\n'.format(bqm_time))

Time used for construction BQM (s): 0.000



### Local heuristic solver

In [65]:
num_reads=5000
t1 = time.time()
sampleset5 = SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)
print('Time used (s): {:.3f}\n'.format(heur_time))

print('Lowest energy should be nearly:',-max_count)  
print('Lowest energy reached:',int(sampleset5.first.energy))
print('Lowest energy occurences: {:.1f} %'.format(sampleset5.first.num_occurrences/num_reads*100))

Time used (s): 3.966

Lowest energy should be nearly: -191
Lowest energy reached: -181
Lowest energy occurences: 4.4 %


In [66]:
for r in result_info(sampleset5,s,t,E):
    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

Route 0-9: 0-13-11-3-9, weight 10
Route 0-9: 0-14-9, weight 10
Route 0-9: 0-10-9, weight 10


### Quantum solver

In [67]:
print('Number of logical qubits needed:',Q.shape[0])
print('Number of logical couplers needed:', len(bqm.quadratic))

Number of logical qubits needed: 40
Number of logical couplers needed: 220


In [68]:
from minorminer.busclique import find_clique_embedding

num_reads = 1000

embedding = find_clique_embedding(bqm.variables, machine.to_networkx_graph())  
num_qubits_needed = sum(len(chain) for chain in embedding.values())
print('Estimate actual qubits needed:',num_qubits_needed)

anneal_schedule = [[0.0, 0.0], [40.0, 0.4], [1040.0, 0.4], [1042, 1.0]]
estimated_runtime = machine.solver.estimate_qpu_access_time(num_qubits_needed, num_reads=num_reads, anneal_schedule=anneal_schedule)    
print("Estimate QPU time needed (ms): {:.0f}".format(estimated_runtime/1000, machine.solver.name))

Estimate actual qubits needed: 192
Estimate QPU time needed (ms): 1253


In [69]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [70]:
sampleset6 = EmbeddingComposite(machine).sample(bqm, num_reads=num_reads)

In [71]:
qtime = sampleset6.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset6.info['embedding_context']['embedding'].values())
print('Lowest energy should be nearly:',-max_count)  
print('Lowest energy reached:',int(sampleset6.first.energy))
print('Lowest energy occurences: {:.1f} %'.format(sampleset6.first.num_occurrences/num_reads*100))
print('QPU time used (ms): {:.1f}'.format(qtime))
print('Physical qubits used: {}'.format(qubits))

Lowest energy should be nearly: -191
Lowest energy reached: -175
Lowest energy occurences: 0.1 %
QPU time used (ms): 116.6
Physical qubits used: 88


In [72]:
for r in result_info(sampleset6.aggregate(),s,t,E):
    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

Route 0-9: 0-8-13-4-11-3-9, weight 16
